In [1]:
from protein_engineering.pipeline import AADataset
from torch_geometric.loader import DataLoader as geom_DataLoader
from protein_engineering.archive.test import GB1Olson 

# Data loading
data = GB1Olson()
protein = '2GI9'
dataset = AADataset(protein, code=protein)
dataloader = geom_DataLoader(dataset)


/Users/antoniaboca/miniconda3/envs/diss/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['Mut1 WT amino acid', 'Mut1 Position', 'Mut1 Mutation',
       'Mut2 WT amino acid', 'Mut2 Position', 'Mut2 Mutation', 'Input Count',
       'Selection Count', 'Mut1 Fitness', 'Mut2 Fitness'],
      dtype='object')


In [2]:
import torch
from gvp_antonia.main import ModelWrapper
from protein_engineering.pipeline import Pair, uncertainty_search

# Model loading
MODEL_PATH = '/Users/antoniaboca/partIII-amino-acid-prediction/data/eqgat_debug.ckpt'
best_model = ModelWrapper('eqgat', 1e-3, None, 0.0, n_layers=5)
best_model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu'))['state_dict'])
    

Using Cross Product
With Feature Attention


<All keys matched successfully>

In [4]:
uncertainty_search(best_model, dataloader, k=4, locs=2)

56it [00:04, 12.26it/s]


In [5]:
import pandas as pd
new_df = data.data.copy()
new_df['variant'] = new_df['variant'].astype('str')

mutations = pd.read_csv('mutations.csv')
codes = mutations['mutation_codes']
data.data[new_df['variant'].isin(codes)]

,variant,fitness,sequence,hamming_to_wildtype,is_wildtype,fitness_raw,fitness_percentile,fitness_rank
420,A24D,0.069526,MQYKLILNGKTLKGETTTEAVDADTAEKVFKQYANDNGVDGEWTYD...,1,False,1.072,0.727992,270
421,A24E,1.259880,MQYKLILNGKTLKGETTTEAVDAETAEKVFKQYANDNGVDGEWTYD...,1,False,3.525,0.999674,5
433,A24S,0.735248,MQYKLILNGKTLKGETTTEAVDASTAEKVFKQYANDNGVDGEWTYD...,1,False,2.086,0.989054,27
647,D36A,0.506818,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANANGVDGEWTYD...,1,False,1.660,0.955661,66
657,D36N,0.622188,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANNNGVDGEWTYD...,1,False,1.863,0.981442,38
661,D36S,0.515216,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANSNGVDGEWTYD...,1,False,1.674,0.957404,62
